In [3]:
import os
import openai
import langchain
openai.api_key = ''
os.environ['OPENAI_API_KEY'] = ''

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'Chroma/'

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
loader = [PyPDFLoader('Book.pdf')]
doc = []
for i in loader:
    doc.extend(i.load())

In [17]:
text_splitter = TokenTextSplitter(
    chunk_size = 500,
    chunk_overlap =15
)

In [18]:
splits = text_splitter.split_documents(doc)
embedding = OpenAIEmbeddings()

In [19]:
splits[2].page_content

'Preface\nIntroduction\nThe History of Bhagavad-gîtå\nMa∫galåcara√a\nCHAPTER 1 Sainya-Darçana\nObserving the Armies on the Battlefield\nCHAPTER 2 Så∫khya Yoga  \nThe Yoga of Analysis\nCHAPTER 3 Karma Yoga  \nThe Yoga of Action\nCHAPTER 4 Jñåna Yoga  \nThe Yoga of Knowledge\nCHAPTER 5 Karma-Sannyåsa Yoga  \nThe Yoga of the Renunciation of Action\nCHAPTER 6 Dhyåna Yoga  \nThe Yoga of Meditation\nCHAPTER 7 Jñåna-Vijñåna Yoga\nThe Yoga of Knowledge and RealisationContents\nix\nxi\nxv\nxix\n1\n25\n71\n99\n133\n151\n177'

In [20]:
len(splits)

1099

In [21]:
vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embedding,
    persist_directory = persist_directory
)

In [22]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo-0301"
llm = ChatOpenAI(model_name=llm_name,temperature=0)

In [23]:
from langchain.chains import RetrievalQA

In [30]:
# Build prompt
# Run chain
from langchain.chains import RetrievalQA
question = "what did lord krishna said to arjun about failure? give your response in the form of a tweet"
q2 = "what did lord krishna said to arjun about failure choose from one of the following options: option1 is be positive,option 2 is give up. response only with correct option"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                      )
result = qa_chain({"query": q2})
result["result"]

'Option 2: Lord Krishna did not advise Arjuna to give up in the Bhagavad Gita. Instead, he encouraged him to fulfill his duty as a warrior and not to yield to weakness or unmanliness.'

In [31]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [32]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
while 1>0:
    question = input('enter prompt')
    output = qa(question)
    print(output['answer'])

enter prompt who is the main speaker in gita?


The main speaker in the Bhagavad-gita is Lord Krishna.


enter prompt what were we talking about?


The context does not provide information about the current topic of conversation.


enter prompt what did i just ask?


I'm sorry, I did not ask a previous question. Is there anything else I can assist you with?
